In [1]:
import pandas as pd
import re

def load_and_preprocess_tcd(tcd_filepath):
    df = pd.read_excel(tcd_filepath)

    df = df[["Labels", "Action", "Expected Results", "Description", "link issue Test"]].dropna()

    df["Test_Case_Type"] = df["Labels"].apply(
        lambda x: x.split("_")[-1].strip().lower().replace(" ", "") if isinstance(x, str) else "")
    df["Sub_Feature"] = df["Labels"].apply(
        lambda x: "_".join(x.split("_")[2:-1]).strip() if isinstance(x, str) else "")
    df["Normalized_Feature"] = df["Sub_Feature"].apply(
        lambda x: re.sub(r"[_\s]+", "_", x.strip().lower()) if isinstance(x, str) else "")
    # Clean up link issue test format (convert multiline or semicolon to single underscore-separated)
    df["link issue Test"] = df["link issue Test"].astype(str)
    df["link issue Test"] = df["link issue Test"].str.replace(r"[\n\r;]+", "_", regex=True)
    df["link issue Test"] = df["link issue Test"].str.strip("_")


    return df

def extract_steps(row):
    action_steps = []
    expected_steps = {}

    action_lines = row["Action"].split("\n")
    extracting = False
    for line in action_lines:
        if "Steps:" in line:
            extracting = True
            continue
        if extracting and line.strip():
            clean_line = re.sub(r"^\d+\.\s*", "", line.strip())
            action_steps.append(clean_line)

    expected_lines = row["Expected Results"].split("\n")
    for line in expected_lines:
        if line.strip() and re.match(r"^\d+\.", line):
            parts = line.split(".", 1)
            try:
                step_num = int(parts[0].strip())
                clean_value = parts[1].strip()
                expected_steps[step_num] = clean_value
            except ValueError:
                continue

    final_steps = []
    used_expected = set()

    for i, action in enumerate(action_steps, start=1):
        final_steps.append(action)
        if i in expected_steps:
            final_steps.append(expected_steps[i])
            used_expected.add(i)

    for key in sorted(expected_steps.keys()):
        if key not in used_expected:
            final_steps.append(expected_steps[key])

    return final_steps

def build_robot_test_cases(df):
    robot_structure = {}

    for _, row in df.iterrows():
        feature = row["Sub_Feature"].strip().replace(" ", "_")
        test_case_type = row["Test_Case_Type"].capitalize()
        feature_group = f"{feature}_{test_case_type}"
        documentation = row["link issue Test"]

        test_case = {
            "name": row.get("Description", "").strip(),
            "documentation": documentation,
            "tags": feature,
            "description": row.get("Description", "").strip(),
            "feature": feature,
            "feature_group": feature_group,
            "steps": extract_steps(row)
        }

        if feature not in robot_structure:
            robot_structure[feature] = []

        robot_structure[feature].append(test_case)

    return robot_structure


def generate_robot_test_cases(df, output_path, keyword_mapping_df=None, header_file_path=None):
    category_order = ["logicalcombination", "failuremodes", "powermodes", "configuration", "voltagemodes"]

    # Build keyword mapping if provided
    keyword_map = {}
    if keyword_mapping_df is not None:
        keyword_map = dict(zip(
            keyword_mapping_df["TCD Keywords"].str.strip().str.lower(),
            keyword_mapping_df["Python Func Name"].str.strip()
        ))

    with open(output_path, "w") as f:
       
        if header_file_path:
            with open(header_file_path, "r") as header_file:
                header_content = header_file.read()
                f.write(header_content.strip() + "\n\n")
        else:
            raise ValueError("Header file path must be provided.")

        f.write("*** Test Cases ***\n\n")

        
        feature_groups = df.groupby("Normalized_Feature")

        for _, group_df in feature_groups:
            tc_index = 1 
            for _, row in group_df.iterrows():
                test_name = row.get("Description", "Unnamed Test Case")
                feature_tag = row.get("Sub_Feature", "").strip()
                category = row.get("Test_Case_Type", "").strip().lower()
                full_group = f"{feature_tag}_{category}"
                linked_issues = row.get("link issue Test", "")

                
                f.write(f"TC{tc_index:03d}: [SYS5] {test_name}\n")
                tc_index += 1

                f.write(f"    [Documentation]    {linked_issues}\n")
                f.write(f"    [Tags]    {feature_tag}\n")
                f.write(f"    [Description]    {test_name}\n")
                f.write(f"    [Feature]    {feature_tag}\n")
                f.write(f"    [Feature_group]    {full_group}\n\n")

                steps = row.get("Steps", [])
                for step in steps:
                    if ":" in step:
                        clean_step = re.sub(r"^\d+\.\s*", "", step.strip())
                        keyword, value = clean_step.split(":", 1)
                        keyword_clean = keyword.strip().lower()
                        value_clean = value.strip()

                        #  Handle SetVoltage values
                        if keyword_clean == "setvoltage":
                            if value_clean == "7":
                                f.write("    Set Low Voltage\n")
                            elif value_clean == "13":
                                f.write("    Default Recovery Voltage\n")
                            elif value_clean == "18":
                                f.write("    Set High Voltage\n")
                            continue

                        # ✅ Handle RELAY_ON and RELAY_OFF values
                        if keyword_clean == "relay_off":
                            if value_clean.lower() == "ign":
                                f.write("    Do IGN OFF\n")
                                continue
                            elif value_clean.lower() == "bat":
                                f.write("    Do BAT OFF\n")
                                continue
                        if keyword_clean == "relay_on":
                            if value_clean.lower() == "ign":
                                f.write("    Do IGN ON\n")
                                continue
                            elif value_clean.lower() == "bat":
                                f.write("    Do BAT ON\n")
                                continue

                        # ✅ Replace keyword if found in mapping
                        replacement_keyword = keyword_map.get(keyword_clean, keyword.strip())
                        f.write(f"    {replacement_keyword}    {value_clean}\n")

                f.write("\n")


In [ ]:
tcd_file_path = "test_2_multiple_features.xlsx"                     
keyword_mapping_path = "Keyword_TCD_latest.xlsx"    
header_file_path = "header.txt"            
output_robot_file = "final_test_output.robot"    

# === Load files ===
df = load_and_preprocess_tcd(tcd_file_path)

# Add Steps column to df (important!)
df["Steps"] = df.apply(lambda row: extract_steps(row), axis=1)

# Load keyword mapping if exists
try:
    keyword_df = pd.read_excel(keyword_mapping_path)
except FileNotFoundError:
    keyword_df = None

# Generate .robot file
generate_robot_test_cases(df, output_robot_file, keyword_mapping_df=keyword_df, header_file_path=header_file_path)

print("✅ .robot file generated successfully!")

FileNotFoundError: [Errno 2] No such file or directory: 'test_2_multiple_features.xlsx'